observemos para qué características se presenta información faltante

In [6]:
import pandas as pd
# Cargar el dataset completo
dataset = pd.read_csv('loan.csv')  # Reemplaza con el nombre de tu archivo

In [ ]:

pd.set_option('display.max_rows', 76)



# Calcular el porcentaje de entradas no nulas para cada atributo
attribute_completeness = dataset.notnull().mean() * 100

# Convertir los resultados en un DataFrame para facilitar su lectura
attribute_completeness_df = attribute_completeness.reset_index()
attribute_completeness_df.columns = ['Attribute', 'Completeness (%)']

# Ordenar por porcentaje de completitud
attribute_completeness_df = attribute_completeness_df.sort_values(by='Completeness (%)', ascending=False)

# Mostrar el resultado
print(attribute_completeness_df)

# Guardar los resultados en un archivo CSV (opcional)
attribute_completeness_df.to_csv('attribute_completeness.csv', index=False)


se eliminan las columnas para las caracteristicas con una presencia menor o igual al 25%:

In [ ]:
# Filtrar columnas con más del 25% de datos presentes
columns_to_keep = attribute_completeness[attribute_completeness > 25].index

# Crear un nuevo dataset con las columnas filtradas
filtered_dataset = dataset[columns_to_keep]

# Guardar el dataset filtrado en un nuevo archivo CSV
filtered_dataset.to_csv('loan.csv', index=False)

# Eliminar las columnas directamente en el dataset

dataset = pd.read_csv('loan.csv')
# Mostrar las columnas eliminadas
removed_columns = attribute_completeness[attribute_completeness <= 25].index
print(f"Columnas eliminadas: {list(removed_columns)}")

In [ ]:

pd.set_option('display.max_rows', 76)

# Calcular el porcentaje de entradas no nulas para cada atributo
attribute_completeness = dataset.notnull().mean() * 100

# Convertir los resultados en un DataFrame para facilitar su lectura
attribute_completeness_df = attribute_completeness.reset_index()
attribute_completeness_df.columns = ['Attribute', 'Completeness (%)']

# Ordenar por porcentaje de completitud
attribute_completeness_df = attribute_completeness_df.sort_values(by='Completeness (%)', ascending=False)


# Mostrar el resultado
print(attribute_completeness_df)




se eliminan caracteristicas que por su descripción o caracteristicas no son útiles para el modelo




## Lista de Columnas y Motivos de Eliminación

### 1. `addr_state`
- **Razón de eliminación**:
  - Representa el estado de residencia del cliente.
  - Es una característica con baja correlación con la probabilidad de incumplimiento, ya que las tasas de incumplimiento no suelen depender significativamente de la ubicación geográfica a este nivel.

### 2. `collection_recovery_fee`
- **Razón de eliminación**:
  - Representa las tarifas asociadas con la recuperación de préstamos después de un incumplimiento.
  - **Problema principal**: Esta información solo está disponible después de que el evento de incumplimiento ha ocurrido, lo que introduce un sesgo temporal y haría que el modelo dependa de información futura.

### 3. `desc`
- **Razón de eliminación**:
  - Contiene descripciones textuales proporcionadas por los clientes.
  - Es irrelevante para predecir el incumplimiento porque no aporta información estructurada y es costoso procesarlo como texto libre.

### 4. `title`
- **Razón de eliminación**:
  - Similar a `desc`, es un campo textual relacionado con el propósito del préstamo.
  - Es redundante con la columna `purpose`, que ya clasifica el propósito del préstamo de forma estructurada.

### 5. `zip_code`
- **Razón de eliminación**:
  - Representa el código postal del cliente.
  - Tiene una alta cardinalidad y una correlación débil con el incumplimiento, lo que lo hace irrelevante para el modelo.

### 6. `policy_code`
- **Razón de eliminación**:
  - Es un código interno utilizado para clasificar las políticas del prestamista.
  - Es irrelevante para la predicción del incumplimiento porque no aporta información sobre el comportamiento del cliente.

### 7. `member_id`
- **Razón de eliminación**:
  - Es un identificador único asignado a cada cliente.
  - No tiene valor predictivo porque no aporta información sobre las características del cliente o su comportamiento financiero.

### 8. `id`
- **Razón de eliminación**:
  - Similar a `member_id`, es un identificador único del préstamo.
  - Es irrelevante y no tiene correlación con el incumplimiento.

### 9. `url`
- **Razón de eliminación**:
  - Contiene enlaces a detalles del préstamo.
  - Es completamente irrelevante para el modelo y no tiene ningún valor predictivo.

### 10. `emp_title`
- **Razón de eliminación**:
  - Representa el título del empleo del cliente.
  - Tiene **alta cardinalidad** (más de 300.000 trabajos diferentes ), lo que dificulta su representación de manera eficiente sin aumentar demasiado la dimensionalidad del modelo.


### 11. `pymnt_plan`
- **Razón de eliminación**:
  - Representa si el cliente está en un plan de pagos.
  - Tiene **muy poca variabilidad**, ya que casi todas las entradas tienen el valor `"n"` (no) y solo unas pocas tienen el valor `"y"` (sí) (10). además mo se observa correlación con loan_status


### 12. `recoveries`
- **Razón de eliminación**:
  - Representa los montos recuperados después de un incumplimiento.
  - **Problema principal**: Esta información solo está disponible después de que el incumplimiento ha ocurrido, lo que introduce un sesgo temporal y no es útil para predecir incumplimientos futuros.

---


# Razones para Eliminar Columnas Basadas en Fechas

## Actualización de las Razones de Eliminación para las Fechas

### 1. `next_pymnt_d`

  - **Problema principal**:
    - Si la base de datos contiene datos históricos y `next_pymnt_d` se refiere a fechas que ya han pasado, el modelo podría comportarse de manera inconsistente:
      - **Antes de esa fecha**: El modelo podría asumir que el cliente está al día con los pagos.
      - **Después de esa fecha**: El modelo podría asumir que el cliente está en incumplimiento.
    - Esto introduce un problema de dependencia temporal, donde el modelo está influenciado por eventos futuros o pasados de manera incoherente.

### 2. `issue_d`
- **Razón de eliminación**:
  - Representa la fecha en la que se emitió el préstamo.
  - **Problema principal**:
    - En datasets históricos, esta fecha puede ser demasiado antigua, y los patrones asociados a esa antigüedad pueden no ser relevantes para las predicciones actuales.


### 3. `earliest_cr_line`
- **Razón de eliminación**:
  - Representa la fecha de apertura de la primera línea de crédito del cliente.
  - **Problema principal**:
    - Similar a `issue_d`, esta información podría estar demasiado influenciada por la antigüedad del dataset.


### 4. `last_pymnt_d`
- **Razón de eliminación**:
  - Representa la fecha del último pago realizado por el cliente.
    - Esto introduce inconsistencia en el comportamiento del modelo dependiendo del momento en el que se entrena.

### 5. `last_credit_pull_d`
- **Razón de eliminación**:
  - Representa la última fecha en que el prestamista verificó el historial crediticio del cliente.
  - **Problema principal**:
    - Si el dataset contiene datos antiguos, el modelo podría interpretar que clientes con verificaciones más recientes tienen un menor riesgo de incumplimiento, lo cual no siempre es cierto.
    - Este tipo de columnas puede sesgar las predicciones dependiendo de la antigüedad del dataset.

---



In [ ]:
# Lista de columnas a eliminar
columns_to_remove = [
    'addr_state', 
    'collection_recovery_fee',  #solo está disponible después de que el evento de incumplimiento haya ocurrido#
    'desc',  #irrelevante
    'title',  #irrelevante
    'zip_code',  #irrelevante
    'policy_code', 
    'member_id', #irrelevante
    'id', #irrelevante
    'url', #irrelevante
    'next_pymnt_d', #posibles problemas debido a fechas
    'issue_d',  #posibles problemas debido a fechas
    'emp_title',  # alta cardinalidad
    'earliest_cr_line', #posibles problemas debido a fechas
    'pymnt_plan', # poca variabilidad, solo 10 entradas en para "y", el resto para "n", poca correlacion con objetivo
    'last_pymnt_d', #posibles problemas debido a fechas
    'last_credit_pull_d', #posibles problemas debido a fechas
    'recoveries', #  solo está disponible después de que el evento de incumplimiento haya ocurrido
    'id',
 
    

]

# Eliminar las columnas especificadas directamente en el dataset filtrado
dataset.drop(columns=columns_to_remove, errors='ignore', inplace=True)

# Guardar el dataset sobrescrito en el mismo archivo
dataset.to_csv('loan.csv', index=False)

# Mostrar las columnas eliminadas
print(f"Columnas eliminadas: {columns_to_remove}")
print("El dataset ha sido sobrescrito en 'loan.csv'")


analisis de correlacion entre variables

In [ ]:
import pandas as pd
dataset = pd.read_csv("loan.csv")
# Cargar el dataset


# Filtrar solo columnas numéricas
numeric_dataset = dataset.select_dtypes(include=['number'])

# Calcular la matriz de correlación para las variables numéricas
correlation_matrix = numeric_dataset.corr()

# Convertir la matriz de correlación en formato largo (lista de pares de variables y sus correlaciones)
correlation_pairs = correlation_matrix.unstack().reset_index()
correlation_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# Filtrar correlaciones redundantes y mantener valores únicos
correlation_pairs = correlation_pairs[correlation_pairs['Variable 1'] != correlation_pairs['Variable 2']]
correlation_pairs = correlation_pairs.drop_duplicates(subset=['Correlation'], keep='first')

# Filtrar correlaciones según un umbral definido por el usuario
threshold = 0.85  # Cambia este valor según tu necesidad
filtered_correlations = correlation_pairs[correlation_pairs['Correlation'].abs() >= threshold]

# Mostrar la tabla de correlaciones filtradas
print(filtered_correlations)


### Evaluación de características altamente correlacionadas

A continuación, se presenta la selección de características basadas en las correlaciones y la decisión de descartar las siguientes variables: `funded_amnt`, `funded_amnt_inv`, `out_prncp_inv`, `total_pymnt_inv`, `total_pymnt`, y `loan_amnt`. Se justifican las decisiones de mantener o descartar cada variable.

#### 1. **loan_amnt vs funded_amnt (correlación 0.999263)**
   - **Descripción**: `loan_amnt` es el monto solicitado, mientras que `funded_amnt` es el monto financiado.
   - **Selección**: Se descarta **ambas variables**, ya que su información es redundante con otras como `installment`, que combina monto, plazo y tasa de interés, y tiene mayor valor explicativo.

#### 2. **loan_amnt vs funded_amnt_inv (correlación 0.997115)**
   - **Descripción**: `funded_amnt_inv` es la cantidad financiada efectivamente por los inversores.
   - **Selección**: Se descarta `funded_amnt_inv` por ser dependiente del monto aprobado y no aportar información adicional más allá de otras variables seleccionadas.

#### 3. **loan_amnt vs installment (correlación 0.944977)**
   - **Descripción**: `installment` refleja el pago mensual calculado a partir de `loan_amnt`, el plazo y la tasa de interés.
   - **Selección**: Se mantiene **`installment`** porque captura tanto el monto como las condiciones del préstamo, haciendo que sea más representativa de la carga financiera del prestatario.

#### 4. **funded_amnt vs funded_amnt_inv (correlación 0.998025)**
   - **Selección**: Ambas variables son redundantes y se descartan. Información relevante ya está representada por `installment`.

#### 5. **funded_amnt vs installment (correlación 0.946005)**
   - **Selección**: Se descarta `funded_amnt` en favor de `installment` por las razones mencionadas.

#### 6. **out_prncp vs out_prncp_inv (correlación 0.999997)**
   - **Descripción**: Ambas miden el saldo del principal pendiente. `out_prncp` representa el monto total, mientras que `out_prncp_inv` es la parte correspondiente a los inversores.
   - **Selección**: Se mantiene **`out_prncp`**, ya que es más general y captura la perspectiva del prestatario, mientras que `out_prncp_inv` se descarta.

#### 7. **total_pymnt vs total_pymnt_inv (correlación 0.997592)**
   - **Descripción**: `total_pymnt` refleja el total pagado por el prestatario, mientras que `total_pymnt_inv` incluye lo recibido por los inversores.
   - **Selección**: Se descartan ambas variables, ya que su información es redundante con otras métricas como `installment` y `total_rec_prncp`.

#### 8. **total_pymnt vs total_rec_prncp (correlación 0.970043)**
   - **Descripción**: `total_rec_prncp` refleja el total de principal recuperado por el prestatario.
   - **Selección**: Se mantiene **`total_rec_prncp`**, ya que mide la capacidad de pago del prestatario y es más relevante para la evaluación del riesgo.

#### 9. **installment vs total_pymnt (correlación 0.862285)**
   - **Selección**: Se prioriza `installment` porque mide directamente la carga financiera mensual del prestatario.

### Resumen de variables seleccionadas:
Se mantiene la información clave para evaluar la capacidad de pago y el riesgo del prestatario:
- **installment**: Representa la carga financiera real mensual.
- **out_prncp**: Saldo del principal pendiente, relevante para medir el riesgo residual.
- **total_rec_prncp**: Principal efectivamente recuperado, que refleja el desempeño del prestatario en el pago de la deuda.

### Variables descartadas:
- **`loan_amnt`**: Representa la intención inicial del prestatario, pero su información está reflejada en `installment`.
- **`funded_amnt` y `funded_amnt_inv`**: Redundantes con `installment`.
- **`out_prncp_inv`**: Subconjunto de `out_prncp`.
- **`total_pymnt` y `total_pymnt_inv`**: Redundantes con `installment` y `total_rec_prncp`.


In [ ]:
# Lista de columnas a eliminar
columns_to_remove = [
    'funded_amnt', 
    'funded_amnt_inv', 
    'out_prncp_inv', 
    'total_pymnt_inv',
    'total_pymnt',
    'loan_amnt'
]

# Eliminar las columnas directamente en el dataset
dataset.drop(columns=columns_to_remove, errors='ignore', inplace=True)

# Guardar el dataset modificado en el mismo archivo
dataset.to_csv('loan.csv', index=False)

print(f"Columnas eliminadas: {columns_to_remove}")
print("El dataset modificado ha sido sobrescrito en 'loan.csv'")


Se revisa nuevamente el porcentaje de completitud de cada característica en el dataset para revisar información faltante

In [ ]:
attribute_completeness1 = dataset.notnull().mean() * 100

# Convertir los resultados en un DataFrame para facilitar su lectura
attribute_completeness_df1 = attribute_completeness1.reset_index()
attribute_completeness_df1.columns = ['Attribute', 'Completeness (%)']

# Ordenar por porcentaje de completitud
attribute_completeness_df1 = attribute_completeness_df1.sort_values(by='Completeness (%)', ascending=False)
print(attribute_completeness_df1)

se observa que la característica  mths_since_last_delinq es la única con mucha información faltante, así que se decide retirar ya que no es claro si un valor NaN significa 0, se retira para no correr riesgos.

In [ ]:
# Lista de columnas a eliminar
columns_to_remove = [
    'mths_since_last_delinq'
]

# Eliminar las columnas directamente en el dataset
dataset.drop(columns=columns_to_remove, errors='ignore', inplace=True)

# Guardar el dataset modificado en el mismo archivo
dataset.to_csv('loan.csv', index=False)

print(f"Columnas eliminadas: {columns_to_remove}")
print("El dataset modificado ha sido sobrescrito en 'loan.csv'")


después de revisar las variables categoricas, se decide modificar emp_length para ser numérica

In [ ]:
import pandas as pd

# Cargar el dataset

# Función corregida para convertir la columna emp_length a enteros
def emp_length_converter(df, column):
    df[column] = df[column].str.replace(r'\+ years', '', regex=True)  # Eliminar '+ years'
    df[column] = df[column].str.replace(r'< 1 year', '0', regex=True)  # Reemplazar '< 1 year' por '0'
    df[column] = df[column].str.replace(r' years', '', regex=True)    # Eliminar ' years'
    df[column] = df[column].str.replace(r' year', '', regex=True)     # Eliminar ' year'
    df[column] = df[column].str.replace(r'\+', '', regex=True)        # Eliminar cualquier '+'
    df[column] = pd.to_numeric(df[column], errors='coerce')           # Convertir a numérico (NaN si falla)


# Aplicar la conversión
emp_length_converter(dataset, 'emp_length')


# Guardar el dataset modificado
dataset.to_csv('loan.csv', index=False)
print("El dataset modificado ha sido guardado como 'loan.csv'.")


### Clasificación de `loan_status` basada en conocimiento del dominio

Con base en el conocimiento del dominio, clasificaremos los préstamos según los siguientes valores de `loan_status` como incumplidos (o 0):

- **Charged Off**
- **Default**
- **Late (31–120 days)**
- **Does not meet the credit policy. Status:Charged Off**

Todos los demás valores serán clasificados como buenos (o 1).


In [ ]:


#

# Función para transformar la columna 'loan_status'
def transform_loan_status(df, column):
    # Lista de valores que se consideran "en incumplimiento" (0)
    default_statuses = [
        "Charged Off",
        "Default",
        "Late (31-120 days)",
        "Does not meet the credit policy. Status:Charged Off"
    ]
    
    # Transformar la columna
    df[column] = df[column].apply(lambda x: 0 if x in default_statuses else 1)
    
    return df

# Aplicar la transformación
dataset = transform_loan_status(dataset, 'loan_status')

# Verificar los cambios
print(dataset['loan_status'].value_counts())

# Guardar el dataset modificado
dataset.to_csv('loan.csv', index=False)
print("El dataset modificado ha sido guardado como 'loan.csv'.")


se lleva a cabo un test de chi cuadrado para identificar las variables categóricas más relevantes.
todas tienen valor P muy bajo, así que se opta por eliminar grade, el cual está contenido en subgrade. 

In [ ]:
import pandas as pd
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder



# Specify the target column (binary classification)
target_column = 'loan_status'  # Replace with the actual name of your target column

# Ensure the target column is binary
if len(dataset[target_column].unique()) != 2:
    raise ValueError("The target column is not binary. Ensure it contains only two unique values.")

# Identify categorical features
categorical_columns = dataset.select_dtypes(include=['object']).columns.tolist()

# Remove the target column from the list of categorical features (if it's included)
if target_column in categorical_columns:
    categorical_columns.remove(target_column)

# Encode categorical features and target variable
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(dataset[column].astype(str))
    label_encoders[column] = le

# Encode the target column if necessary
if dataset[target_column].dtype == 'object':
    dataset[target_column] = LabelEncoder().fit_transform(dataset[target_column])

# Perform Chi-squared test
chi2_values, p_values = chi2(dataset[categorical_columns], dataset[target_column])

# Create a DataFrame with the results
chi2_results = pd.DataFrame({
    'Feature': categorical_columns,
    'Chi2 Value': chi2_values,
    'P-Value': p_values
}).sort_values(by='Chi2 Value', ascending=False)

# Display the results
print(chi2_results)

# Save the results to a CSV file
chi2_results.to_csv('chi2_feature_selection.csv', index=False)
print("Chi-squared test results saved to 'chi2_feature_selection.csv'.")


In [42]:
dataset.drop(columns=['grade'], inplace=True)
print("La columna 'grade' ha sido eliminada.")

La columna 'grade' ha sido eliminada.


ahora se eliminan las filas que tengan al menos un campo con información faltante

In [ ]:
# Eliminar las filas con al menos un NaN

dataset = dataset.dropna()
print(f"Total de filas después de eliminar las filas con NaN: {dataset.shape[0]}")

  #  Guardar el dataset limpio s
dataset.to_csv('cleaned_dataset.csv', index=False)
print("El dataset limpio ha sido guardado como 'cleaned_dataset.csv'.")

Total de filas después de eliminar las filas con NaN: 773968
El dataset limpio ha sido guardado como 'cleaned_dataset.csv'.


In [2]:
import pandas as pd
dataset = pd.read_csv("cleaned_dataset.csv")

In [3]:
print(dataset.dtypes)

term                           object
int_rate                      float64
installment                   float64
sub_grade                      object
emp_length                    float64
home_ownership                 object
annual_inc                    float64
verification_status            object
loan_status                     int64
purpose                        object
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
revol_util                    float64
total_acc                     float64
initial_list_status            object
out_prncp                     float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
last_pymnt_amnt               float64
collections_12_mths_ex_med    float64
application_type               object
acc_now_deli